In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00


In [ ]:
from typing import List
import numpy as np
import torch
import evaluate
from sklearn.model_selection import train_test_split
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [ ]:
tagged_sentences = nltk.corpus.treebank.tagged_sents()
print('Number of samples: ', len(tagged_sentences))

Number of samples:  3914


In [ ]:
sentences, sentence_tags = [], []
for tagged_sentence in tagged_sentences:
  sentence, tags = zip(*tagged_sentence)
  sentences.append([word.lower() for word in sentence])
  sentence_tags.append([tag for tag in tags])

In [ ]:
train_sentences, test_sentences, train_tags, test_tags = train_test_split(
    sentences,
    sentence_tags,
    test_size=0.3
)
test_sentences, valid_sentences, test_tags, valid_tags = train_test_split(
    test_sentences,
    test_tags,
    test_size=0.5
)

In [ ]:
from transformers import AutoTokenizer
from torch.utils.data import Dataset

model_name = "QCRI/bert-base-multilingual-cased-pos-english"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
)
MAX_LEN = max(len(sentence) for sentence in train_sentences)

class PosTagging_Dataset(Dataset):
  def __init__(self,
               sentences: List[List[str]],
               tags: List[List[str]],
               tokenizer,
               label2id,
               max_len=MAX_LEN):
    self.sentences = sentences
    self.tags = tags
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.label2id = label2id

  def __len__(self):
    return len(self.sentences)

  def __getitem__(self, idx):
    input_token = self.sentences[idx]
    label_token = self.tags[idx]

    input_token = self.tokenizer.convert_tokens_to_ids(input_token)
    attention_mask = [1] * len(input_token)
    labels = [self.label2id[token] for token in label_token]

    return{
        'input_ids': self.pad_and_truncate(input_token, pad_id=self.tokenizer.pad_token_id),
        'labels': self.pad_and_truncate(labels, pad_id=self.label2id['0']),
        'attention_mask': self.pad_and_truncate(attention_mask, pad_id=0)
    }


  def pad_and_truncate(self, inputs: List[int], pad_id: int):
    if len(inputs) < self.max_len:
      padded_inputs = inputs + [pad_id] * (self.max_len - len(inputs))
    else:
      padded_inputs = inputs[:self.max_len]

    return torch.as_tensor(padded_inputs)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(model_name)

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from collections import defaultdict
label2id = defaultdict(int, model.config.label2id)
id2label = {id: label for label, id in label2id.items()}

In [ ]:
train_dataset = PosTagging_Dataset(train_sentences, train_tags, tokenizer, label2id)
val_dataset = PosTagging_Dataset(valid_sentences, valid_tags, tokenizer, label2id)
test_dataset = PosTagging_Dataset(test_sentences, test_tags, tokenizer, label2id)

In [ ]:
accuracy = evaluate.load('accuracy')

ignore_label = len(label2id)

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  mask = labels != ignore_label
  predictions = np.argmax(predictions, axis=-1)
  return accuracy.compute(predictions=predictions[mask], references=labels[mask])

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='out_dir',
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.036982,0.989212
2,No log,0.032896,0.990423
3,0.034800,0.031277,0.990982
4,0.034800,0.030582,0.991202
5,0.034800,0.029551,0.991509
6,0.023600,0.029367,0.991679
7,0.023600,0.029651,0.991723
8,0.023600,0.029522,0.991829
9,0.018800,0.029609,0.991804
10,0.018800,0.029598,0.991848


TrainOutput(global_step=1720, training_loss=0.024683894528899082, metrics={'train_runtime': 1762.116, 'train_samples_per_second': 15.544, 'train_steps_per_second': 0.976, 'total_flos': 3789641345256360.0, 'train_loss': 0.024683894528899082, 'epoch': 10.0})

In [ ]:
test_sentences = 'I am Huy from PTIT and I am training in research'
input = torch.as_tensor([tokenizer.convert_tokens_to_ids(test_sentences.split())])
input = input.to('cuda')

outputs = model(input)
_, preds = torch.max(outputs.logits, -1)
preds = preds[0].cpu().numpy()

pred_tags = ''
for pred in preds:
  pred_tags += id2label[pred] + ' '
pred_tags

'PRP VBP NNP IN NNP CC PRP VBP VBG IN NN '